Pretrained Models

Pretrained models in deep learning are trained on large datasets. Here are some popular pretrained models:

1. **VGG (Visual Geometry Group) Models:**
   - VGG16
   - VGG19

2. **ResNet (Residual Networks) Models:**
   - ResNet-18
   - ResNet-34
   - ResNet-50
   - ResNet-101
   - ResNet-152

3. **Inception Models:**
   - InceptionV3
   - InceptionResNetV2

4. **MobileNet Models:**
   - MobileNetV1
   - MobileNetV2
   - MobileNetV3

5. **DenseNet Models:**
   - DenseNet-121
   - DenseNet-169
   - DenseNet-201

6. **EfficientNet Models:**
   - EfficientNetB0, B1, B2, B3, B4, B5, B6, B7

7. **Xception (Extreme Inception) Model**

8. **ResNeXt Models:**
   - ResNeXt-50
   - ResNeXt-101

9. **SqueezeNet Model**

10. **NASNet (Neural Architecture Search Network) Models:**
    - NASNetLarge
    - NASNetMobile

11. **SENet (Squeeze-and-Excitation Networks) Models:**
    - SENet-154
    - SENet-50

12. **ShuffleNet Models:**
    - ShuffleNetV1
    - ShuffleNetV2



Choosen Models: "a5tarthom according to my preferences"

VGG19
ResNet-50
InceptionV3
MobileNetV3 "MobileNetV3 is not directly available in torchvision"
DenseNet-169

VGG19

In [ ]:
# Import necessary PyTorch modules
import torch
import torch.nn as nn
import torchvision.models as models


# Define a class for VGG19 without the last layer
class VGG19WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG19WithoutLastLayer, self).__init__()

        # Load the pretrained VGG19 model from torchvision
        vgg19 = models.vgg19(pretrained=True)

        # Extract the feature layers (convolutional and pooling layers)
        self.features = vgg19.features

        # Global average pooling layer
        self.avgpool = vgg19.avgpool

        # Extract the classifier layers, excluding the last fully connected layer
        self.classifier = nn.Sequential(*list(vgg19.classifier.children())[:-1])

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = self.avgpool(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Forward pass through the classifier layers
        x = self.classifier(x)
        return x


# Function to add a custom last layer to the base model
def add_custom_last_layer(base_model, num_classes):
    # Add a linear layer (fully connected) with the specified number of classes
    last_layer = nn.Linear(
        in_features=base_model.classifier[-1].in_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


# Function to train the model
def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage: BUT IN OUR CASE WE WILL USE METHOD WHICH IS IN THE TOOL
# (Assuming you have train_loader and val_loader for your dataset)

# Create VGG19 base model without the last layer
base_model = VGG19WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

ResNet-50

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


class ResNet50WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet50WithoutLastLayer, self).__init__()

        # Load the pretrained ResNet-50 model from torchvision
        resnet50 = models.resnet50(pretrained=True)

        # Extract all layers except the last fully connected layer
        self.features = nn.Sequential(*list(resnet50.children())[:-1])

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = x.view(x.size(0), -1)

        return x


def add_custom_last_layer(base_model, num_classes):
    # Add your custom last layer based on the task (e.g., classification)
    last_layer = nn.Linear(
        in_features=base_model.features[-1][0].in_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create ResNet-50 base model without the last layer
base_model = ResNet50WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

InceptionV3

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


class InceptionV3WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionV3WithoutLastLayer, self).__init__()

        # Load the pretrained InceptionV3 model from torchvision
        inception_v3 = models.inception_v3(pretrained=True, aux_logits=False)

        # Extract all layers except the last fully connected layer
        self.features = nn.Sequential(*list(inception_v3.children())[:-1])

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = x.view(x.size(0), -1)

        return x


def add_custom_last_layer(base_model, num_classes):
    # Add your custom last layer based on the task (e.g., classification)
    last_layer = nn.Linear(
        in_features=base_model.features[-1][-1].in_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create InceptionV3 base model without the last layer
base_model = InceptionV3WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

DenseNet-169

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


class DenseNet169WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(DenseNet169WithoutLastLayer, self).__init__()

        # Load the pretrained DenseNet-169 model from torchvision
        densenet169 = models.densenet169(pretrained=True)

        # Extract all layers except the last fully connected layer
        self.features = densenet169.features

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)

        return x


def add_custom_last_layer(base_model, num_classes):
    # Add your custom last layer based on the task (e.g., classification)
    last_layer = nn.Linear(
        in_features=base_model.features[-1].in_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create DenseNet-169 base model without the last layer
base_model = DenseNet169WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

MobileNetV3 "MobileNetV3 is not directly available in torchvision"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MobileNetV3Block(nn.Module):
    def __init__(
        self, in_channels, out_channels, kernel_size, stride, expansion_factor, se_ratio
    ):
        super(MobileNetV3Block, self).__init__()
        hidden_dim = int(in_channels * expansion_factor)

        self.conv1 = nn.Conv2d(in_channels, hidden_dim, 1, 1, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(hidden_dim)
        self.act1 = nn.ReLU6(inplace=True)

        self.conv2 = nn.Conv2d(
            hidden_dim,
            hidden_dim,
            kernel_size,
            stride,
            kernel_size // 2,
            groups=hidden_dim,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(hidden_dim)
        self.act2 = nn.ReLU6(inplace=True)

        # Squeeze-and-Excitation layer
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(hidden_dim, int(hidden_dim * se_ratio), 1),
            nn.ReLU6(inplace=True),
            nn.Conv2d(int(hidden_dim * se_ratio), hidden_dim, 1),
            nn.Sigmoid(),
        )

        self.conv3 = nn.Conv2d(hidden_dim, out_channels, 1, 1, 0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.act3 = nn.ReLU6(inplace=True)

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act2(x)

        # Squeeze-and-Excitation
        se_weights = self.se(x)
        x = x * se_weights

        x = self.conv3(x)
        x = self.bn3(x)

        # Skip connection
        x += identity
        x = self.act3(x)

        return x


class MobileNetV3(nn.Module):
    def __init__(
        self, num_classes=1000, input_size=224, width_multiplier=1.0, se_ratio=0.25
    ):
        super(MobileNetV3, self).__init__()
        channels = [16, 24, 40, 80, 112, 160, 960]
        last_channels = int(1280 * width_multiplier) if width_multiplier > 1.0 else 1280

        self.features = nn.Sequential(
            nn.Conv2d(3, int(channels[0] * width_multiplier), 3, 2, 1, bias=False),
            nn.BatchNorm2d(int(channels[0] * width_multiplier)),
            nn.ReLU6(inplace=True),
            MobileNetV3Block(
                int(channels[0] * width_multiplier),
                int(channels[1] * width_multiplier),
                3,
                2,
                1,
                se_ratio,
            ),
            MobileNetV3Block(
                int(channels[1] * width_multiplier),
                int(channels[2] * width_multiplier),
                3,
                2,
                6,
                se_ratio,
            ),
            MobileNetV3Block(
                int(channels[2] * width_multiplier),
                int(channels[3] * width_multiplier),
                3,
                2,
                6,
                se_ratio,
            ),
            MobileNetV3Block(
                int(channels[3] * width_multiplier),
                int(channels[4] * width_multiplier),
                3,
                1,
                6,
                se_ratio,
            ),
            MobileNetV3Block(
                int(channels[4] * width_multiplier),
                int(channels[5] * width_multiplier),
                3,
                1,
                6,
                se_ratio,
            ),
            nn.Conv2d(
                int(channels[5] * width_multiplier), last_channels, 1, 1, 0, bias=False
            ),
            nn.BatchNorm2d(last_channels),
            nn.ReLU6(inplace=True),
        )

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Linear(last_channels, num_classes),
            nn.ReLU6(inplace=True),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def add_custom_last_layer(base_model, num_classes):
    # Add your custom last layer based on the task (e.g., classification)
    last_layer = nn.Linear(
        in_features=base_model.classifier[0].in_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create MobileNetV3 base model without the last layer
base_model = MobileNetV3()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)